In [1]:
train_path = r"C:\Users\HP\Documents\SA\hacktrain.csv"
test_path = r"C:\Users\HP\Documents\SA\hacktest.csv"
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
test_ids = test_df["ID"].copy()
print("okay")

NameError: name 'pd' is not defined

In [ ]:
ndvi_cols = [col for col in train_df.columns if '_N' in col]

print("\nData loading section completed. Variables 'train_df', 'test_df', 'test_ids', and 'ndvi_cols' are now available.")


Data loading section completed. Variables 'train_df', 'test_df', 'test_ids', and 'ndvi_cols' are now available.


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Assuming train_df and ndvi_cols are already defined from Step 1.
# If you are running this as a separate block, ensure these variables are available.


# --- Step 2: Preprocessing for Training Data ---
print("\nStep 2: Preprocessing Training Data...")

# Drop 'Unnamed: 0' column if it exists (often created during CSV export/import)
if "Unnamed: 0" in train_df.columns:
    train_df.drop(columns=["Unnamed: 0"], inplace=True)
    print("Dropped 'Unnamed: 0' column from train_df.")

# Separate features (X) and target (y) for training
X_train_raw = train_df[ndvi_cols].copy()
y_train_raw = train_df["class"].copy() # This is your target variable

print(f"Original training features shape (X_train_raw): {X_train_raw.shape}")
print(f"Original training target shape (y_train_raw): {y_train_raw.shape}")


# Handle Missing Values (Imputation)
# Using median strategy as it's robust to outliers
imputer = SimpleImputer(strategy="median") # Initialize the imputer
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_raw), columns=ndvi_cols)
print("Training data: Missing values imputed.")

# Scale Features (Standardization)
# Scales features to have a mean of 0 and a standard deviation of 1
scaler = StandardScaler() # Initialize the scaler
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=ndvi_cols)
print("Training data: Features scaled.")

print("\nPreprocessing for Training Data completed.")
print("Variables 'X_train_raw', 'y_train_raw', 'imputer', 'scaler', 'X_train_imputed', and 'X_train_scaled' are now available.")



Step 2: Preprocessing Training Data...
Dropped 'Unnamed: 0' column from train_df.
Original training features shape (X_train_raw): (8000, 27)
Original training target shape (y_train_raw): (8000,)
Training data: Missing values imputed.
Training data: Features scaled.

Preprocessing for Training Data completed.
Variables 'X_train_raw', 'y_train_raw', 'imputer', 'scaler', 'X_train_imputed', and 'X_train_scaled' are now available.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Assuming X_train_scaled is defined from Step 2.
# Also, ndvi_cols should be defined from Step 1.


# --- Step 3: Feature Engineering for Training Data ---
print("\nStep 3: Feature Engineering for Training Data...")
X_train_features = pd.DataFrame() # DataFrame to hold the new engineered features

# Calculate basic statistics across the 27 NDVI time points for each row
X_train_features['ndvi_mean'] = X_train_scaled.mean(axis=1)
X_train_features['ndvi_std'] = X_train_scaled.std(axis=1)
X_train_features['ndvi_min'] = X_train_scaled.min(axis=1)
X_train_features['ndvi_max'] = X_train_scaled.max(axis=1)
X_train_features['ndvi_range'] = X_train_features['ndvi_max'] - X_train_features['ndvi_min']

# Extract the first and last NDVI values directly
X_train_features['ndvi_first'] = X_train_scaled.iloc[:, 0]
X_train_features['ndvi_last'] = X_train_scaled.iloc[:, -1]

# Calculate NDVI Trend (Slope) using a simple linear regression for each time series
slopes = []
# Create an array representing time points (0 to 26 for 27 NDVI values)
time_points = np.arange(X_train_scaled.shape[1]).reshape(-1, 1)

# Iterate through each row (each sample's NDVI time series)
for index, row in X_train_scaled.iterrows():
    model = LinearRegression() # Create a simple linear regression model
    model.fit(time_points, row.values) # Fit model: NDVI values (y) vs. time point indices (x)
    slopes.append(model.coef_[0]) # The coefficient (coef_[0]) is the slope of the line
X_train_features['ndvi_slope'] = slopes

print("Training data: New features created.")
print(f"Engineered training features shape: {X_train_features.shape}")
print(X_train_features.head()) # Display the first few rows of engineered features

print("\nFeature Engineering for Training Data completed.")
print("Variable 'X_train_features' is now available.")


Step 3: Feature Engineering for Training Data...
Training data: New features created.
Engineered training features shape: (8000, 8)
   ndvi_mean  ndvi_std  ndvi_min  ndvi_max  ndvi_range  ndvi_first  ndvi_last  \
0  -1.483386  1.106149 -4.810609  0.284937    5.095546   -2.403797  -0.874448   
1  -1.614397  0.986777 -4.837167  0.213515    5.050682   -2.405323  -0.836678   
2  -1.382754  1.204176 -4.810100  0.426859    5.236959   -2.667544  -1.618977   
3  -1.547693  1.055514 -4.807409  0.214805    5.022214   -2.660945  -1.402493   
4  -0.980644  0.961793 -3.631267  0.289733    3.921001   -2.176788  -1.624669   

   ndvi_slope  
0    0.017136  
1    0.021835  
2    0.007236  
3    0.010499  
4   -0.036052  

Feature Engineering for Training Data completed.
Variable 'X_train_features' is now available.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Assuming test_df, ndvi_cols, imputer, and scaler are already defined
# from previous steps (Data Loading and Step 2).
# If you are running this as a separate block, ensure these variables are available.


# --- Step 4: Preprocessing and Feature Engineering for Test Data ---
# Apply the SAME transformations (using the imputer and scaler fitted on training data)
print("\nStep 4: Preprocessing & Feature Engineering for Test Data...")

# Drop 'Unnamed: 0' column if it exists in test_df
if "Unnamed: 0" in test_df.columns:
    test_df.drop(columns=["Unnamed: 0"], inplace=True)
    print("Dropped 'Unnamed: 0' column from test_df.")


# Separate features (X) for test set (using the same NDVI columns identified from training data)
X_test_raw = test_df[ndvi_cols].copy()

# Impute Missing Values in Test Data (using the 'imputer' fitted on training data)
X_test_imputed = pd.DataFrame(imputer.transform(X_test_raw), columns=ndvi_cols)
print("Test data: Missing values imputed using training imputer.")

# Scale Features in Test Data (using the 'scaler' fitted on training data)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_imputed), columns=ndvi_cols)
print("Test data: Features scaled using training scaler.")

# Feature Engineering for Test Data (applying the exact same calculations as for training data)
X_test_features = pd.DataFrame()
X_test_features['ndvi_mean'] = X_test_scaled.mean(axis=1)
X_test_features['ndvi_std'] = X_test_scaled.std(axis=1)
X_test_features['ndvi_min'] = X_test_scaled.min(axis=1)
X_test_features['ndvi_max'] = X_test_scaled.max(axis=1)
X_test_features['ndvi_range'] = X_test_features['ndvi_max'] - X_test_features['ndvi_min']
X_test_features['ndvi_first'] = X_test_scaled.iloc[:, 0]
X_test_features['ndvi_last'] = X_test_scaled.iloc[:, -1]

# Re-using the 'time_points' array created earlier for consistency
# if not available, define: time_points = np.arange(X_test_scaled.shape[1]).reshape(-1, 1)
test_slopes = []
for index, row in X_test_scaled.iterrows():
    model = LinearRegression()
    model.fit(time_points, row.values)
    test_slopes.append(model.coef_[0])
X_test_features['ndvi_slope'] = test_slopes

print("Test data: New features created.")
print(f"Engineered test features shape: {X_test_features.shape}")
print(X_test_features.head())

print("\nPreprocessing and Feature Engineering for Test Data completed.")
print("Variable 'X_test_features' is now available.")


Step 4: Preprocessing & Feature Engineering for Test Data...
Dropped 'Unnamed: 0' column from test_df.
Test data: Missing values imputed using training imputer.
Test data: Features scaled using training scaler.
Test data: New features created.
Engineered test features shape: (2845, 8)
   ndvi_mean  ndvi_std  ndvi_min  ndvi_max  ndvi_range  ndvi_first  ndvi_last  \
0  -0.070332  0.955974 -1.782583  1.744096    3.526679    0.703785   1.725019   
1   0.069367  0.866696 -1.660023  1.760748    3.420770    0.598592  -0.703466   
2   0.102925  1.043168 -1.475840  2.072498    3.548338    0.684973  -0.707931   
3  -0.061574  1.030381 -1.804253  1.939781    3.744034    0.545740   1.827161   
4   0.381189  1.030266 -1.642597  2.272905    3.915503    0.727963  -0.496199   

   ndvi_slope  
0    0.008428  
1    0.006632  
2    0.005883  
3    0.014902  
4    0.004962  

Preprocessing and Feature Engineering for Test Data completed.
Variable 'X_test_features' is now available.


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming y_train_raw (your 'class' column from train_df) is already defined
# from previous data loading and preprocessing steps.
# If you are running this as a separate block, ensure y_train_raw is available.


# --- Step 5: Encode Target Labels ---
print("\nStep 5: Encoding Target Labels (for 'class' column)...")
label_encoder = LabelEncoder() # Initialize a LabelEncoder
y_train_encoded = label_encoder.fit_transform(y_train_raw) # Fit and transform the string labels to numbers

print(f"Original classes: {label_encoder.classes_}") # Show the mapping from numbers back to original class names
print(f"Encoded labels for first 5 training samples: {y_train_encoded[:5]}") # Display first few encoded labels

print("\nEncoding of target labels completed.")
print("Variable 'y_train_encoded' is now available.")


Step 5: Encoding Target Labels (for 'class' column)...
Original classes: ['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']
Encoded labels for first 5 training samples: [5 5 5 5 5]

Encoding of target labels completed.
Variable 'y_train_encoded' is now available.


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # This will be used in the next step, but often imported here

# Assuming X_train_features and y_train_encoded are already defined
# from previous steps (Feature Engineering and Target Encoding).


# --- Step 6: Train Logistic Regression Model ---
print("\nStep 6: Training Logistic Regression Model...")
# Split training data into a smaller training set and a validation set
# This helps us evaluate model performance on data it hasn't seen during training
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train_features, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded
)

print(f"Model training data shape: {X_train_split.shape}")
print(f"Model validation data shape: {X_val.shape}")

# Initialize and train the Logistic Regression model
# 'multi_class='multinomial'' is used for classification problems with more than two classes
# 'solver='lbfgs'' is a good general-purpose solver for multinomial logistic regression
# 'max_iter=1000' ensures the model has enough iterations to find the best fit
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
model.fit(X_train_split, y_train_split) # Train the model using the engineered features and encoded labels

print("Logistic Regression model trained successfully!")

print("\nModel training completed.")
print("Variables 'model', 'X_train_split', 'X_val', 'y_train_split', and 'y_val' are now available.")



Step 6: Training Logistic Regression Model...
Model training data shape: (6400, 8)
Model validation data shape: (1600, 8)


D:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression model trained successfully!

Model training completed.
Variables 'model', 'X_train_split', 'X_val', 'y_train_split', and 'y_val' are now available.


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Assuming 'model', 'X_val', and 'y_val' are already defined from Step 6.
# If you are running this as a separate block, ensure these variables are available.


# --- Step 7: Evaluate the Model ---
print("\nStep 7: Evaluating the Model on Validation Set...")
y_val_pred = model.predict(X_val) # Make predictions on the validation set
accuracy = accuracy_score(y_val, y_val_pred) # Calculate the accuracy of predictions
print(f"Validation Accuracy: {accuracy:.4f}") # Print the accuracy score

print("\nModel evaluation completed.")


Step 7: Evaluating the Model on Validation Set...
Validation Accuracy: 0.8481

Model evaluation completed.


In [ ]:
pip install xgboost

In [8]:
%pip install pandas

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression # Needed for the model object
# Assuming 'model', 'X_test_features', 'label_encoder', and 'test_ids' are already defined
# from previous steps (Model Training, Feature Engineering for Test Data, and Data Loading).
# If you are running this as a separate block, ensure these variables are available.


# --- Step 8: Predict on Test Set and Generate Submission File ---
print("\nStep 8: Predicting on Test Set and Generating Submission File...")
test_predictions_encoded = model.predict(X_test_features) # Make predictions on the engineered test data

# Convert numerical predictions back to original class names (e.g., 0 -> 'farm', 1 -> 'forest')
final_test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

# Create the submission DataFrame in the format required by Kaggle
submission_df = pd.DataFrame({
    "ID": test_ids,
    "class": final_test_predictions
})

# Save the submission file to the current directory
# index=False prevents pandas from writing the DataFrame index as a column in the CSV
submission_df.to_csv("submission_output.csv", index=False)

print("Submission file 'submission.csv' created successfully in the current directory!")
print("Here are the first few rows of your generated submission file:")
print(submission_df.head())

print("\n--- All steps completed! ---")

Note: you may need to restart the kernel to use updated packages.


c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe: No module named pip


ModuleNotFoundError: No module named 'pandas'

In [9]:
%pip install pandas

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression # Needed for the model object
# Assuming 'model', 'X_test_features', 'label_encoder', and 'test_ids' are already defined
# from previous steps (Model Training, Feature Engineering for Test Data, and Data Loading).
# If you are running this as a separate block, ensure these variables are available.


# --- Step 8: Predict on Test Set and Generate Submission File ---
print("\nStep 8: Predicting on Test Set and Generating Submission File...")
test_predictions_encoded = model.predict(X_test_features) # Make predictions on the engineered test data

# Convert numerical predictions back to original class names (e.g., 0 -> 'farm', 1 -> 'forest')
final_test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

# Create the submission DataFrame in the format required by Kaggle
submission_df = pd.DataFrame({
    "ID": test_ids,
    "class": final_test_predictions
})

# Save the submission file to the current directory
# index=False prevents pandas from writing the DataFrame index as a column in the CSV
submission_df.to_csv("submission_output.csv", index=False)

print("Submission file 'submission.csv' created successfully in the current directory!")
print("Here are the first few rows of your generated submission file:")
print(submission_df.head())

print("\n--- All steps completed! ---")

Note: you may need to restart the kernel to use updated packages.


c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe: No module named pip


ModuleNotFoundError: No module named 'pandas'